In [1]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
from langgraph.graph.message import AnyMessage, add_messages
import requests
from dotenv import load_dotenv

load_dotenv()




class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [4]:
from langchain_core.tools import tool
from langchain_core.tools import ToolException
from langchain_community.tools import TavilySearchResults
from typing import Literal


@tool("api_call")
def api_call(query: str):
    """Searches for todo lists. Useful for searching for todo lists."""
    try :
        # res = requests.get(f'http://jsonplaceholder.typicode.com/{query}')
        res = requests.get(f'http://jsonplaceholder.typicode.com/todos')
        res = res.json()
        # maybe parse it?
        # res = parse_tickets_from_json(res)
        return res
    except Exception as e:
        raise ToolException(e)


@tool("multiply")
def multiply(x: int, y: int):
    """Multiplies two numbers."""
    return x * y

# web_search = TavilySearchResults(
#     max_results=2,
#     search_depth="advanced",
#     include_answer=True,
#     include_raw_content=True,
# )

@tool("web_search")
def web_search(query: str):
    """
    Searches the web using TavilySearchResults. Useful for performing web searches with advanced settings.
    """
    try:
        search = TavilySearchResults(
            max_results=2,
            search_depth="advanced",
            include_answer=True,
            include_raw_content=True,
        )
        results = search.invoke(query)
        return results
    except Exception as e:
        raise ToolException(e)


@tool("final_answer")
def final_answer(
    introduction: str,
    research_steps: str,
    main_body: str,
    conclusion: str,
    sources: str
):
    """Returns a natural language response to the user in the form of a research
    report. There are several sections to this report, those are:
    - `introduction`: a short paragraph introducing the user's question and the
    topic we are researching.
    - `research_steps`: a few bullet points explaining the steps that were taken
    to research your report.
    - `main_body`: this is where the bulk of high quality and concise
    information that answers the user's question belongs. It is 3-4 paragraphs
    long in length.
    - `conclusion`: this is a short single paragraph conclusion providing a
    concise but sophisticated view on what was found.
    - `sources`: a bulletpoint list provided detailed sources for all information
    referenced during the research process
    """
    if type(research_steps) is list:
        research_steps = "\n".join([f"- {r}" for r in research_steps])
    if type(sources) is list:
        sources = "\n".join([f"- {s}" for s in sources])
    return ""

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather, multiply, api_call, web_search, final_answer]


In [7]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0
)

graph = create_react_agent(llm, tools=tools, checkpointer=memory)

In [15]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

config = {"configurable": {"thread_id": "3"}}
inputs = {"messages": [("user", "what do i have to do today?")]}
print_stream(graph.stream(inputs, config, stream_mode="values"))

================================ Human Message =================================

what do i have to do today?
================================== Ai Message ==================================
Tool Calls:
  api_call (call_CxwkKmmvNO7ZfXrGXseyL57x)
 Call ID: call_CxwkKmmvNO7ZfXrGXseyL57x
  Args:
    query: todo list for today
================================= Tool Message =================================
Name: api_call

[{"userId": 1, "id": 1, "title": "delectus aut autem", "completed": false}, {"userId": 1, "id": 2, "title": "quis ut nam facilis et officia qui", "completed": false}, {"userId": 1, "id": 3, "title": "fugiat veniam minus", "completed": false}, {"userId": 1, "id": 4, "title": "et porro tempora", "completed": true}, {"userId": 1, "id": 5, "title": "laboriosam mollitia et enim quasi adipisci quia provident illum", "completed": false}, {"userId": 1, "id": 6, "title": "qui ullam ratione quibusdam voluptatem quia omnis", "completed": false}, {"userId": 1, "id": 7, "title": "illo